# 한빛출판네트워크 - 새로나온책 10 페이지

In [2]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
page = 1
site = 'https://www.hanbit.co.kr'
url = site + f'/store/books/new_book_list.html?page={page}&brand=&cate1=&cate2=&searchKey=&keyWord='
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>한빛출판네트워크</title>
<link href="https://www.hanbit.co.kr/images/common/hanbit.ico" rel="shortcut icon"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="website" property="og:type"/>
<meta content="한빛출판네트워크" property="og:title"/>
<meta content="더 넓은 세상, 더 나은 미래를 위한 아시아 출판 네트워크 :: 한빛미디어, 한빛아카데미, 한빛비즈, 한빛라이프, 한빛에듀" property="og:description"/>
<meta content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" property="og:image"/>
<meta content="https://www.hanbit.co.kr/store/books/new_book_list.html?page=1&amp;brand=&amp;cate1=&amp;cate2=&amp;searchKey=&amp;keyWord=" property="og:url"/>
<link href="https://www.hanbit.co.kr/store/books/new_book_list.html?page=1&amp;brand=&amp;cate1=&amp;cate2=&amp;searchKey=&amp;keyWord=" rel="canonical">
<meta content="한빛미디어,한빛아카데미,한빛비즈,한빛라이프,한빛에듀,리얼타임,대관서비스,

In [4]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [5]:
# title
li = lis[0]
li.select_one('.book_tit').get_text().strip()

'엑셀이 편해지는 파이썬'

In [6]:
# author
li.select_one('.book_writer').get_text().strip()

'펠릭스 춤슈타인'

In [7]:
# class book_tit의 tag: p에 href

In [8]:
href = li.select_one('.book_tit').find('a')['href']
href

'/store/books/look.php?p_code=B2831223974'

In [9]:
url = site + href
url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B2831223974'

In [10]:
sub_req = requests.get(url)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
#sub_soup

In [11]:
# 가격
price = sub_soup.select_one('.pbr').get_text()
price = int(re.sub('[^0-9]','', price))
price

28000

In [12]:
sub_lis = sub_soup.select('.info_list > li')
sub_lis

[<li><strong>저자 : </strong><span>펠릭스 춤슈타인 </span></li>,
 <li><strong>번역 : </strong><span>한선용 </span></li>,
 <li><strong>출간 : </strong><span>2022-04-20</span></li>,
 <li><strong>페이지 : </strong><span>388 쪽</span></li>,
 <li><strong>ISBN : </strong><span>9791162245545</span></li>,
 <li><strong>물류코드 :</strong><span>10554</span></li>]

In [13]:
for sub_li in sub_lis:
    print(sub_li.get_text())

저자 : 펠릭스 춤슈타인 
번역 : 한선용 
출간 : 2022-04-20
페이지 : 388 쪽
ISBN : 9791162245545
물류코드 :10554


In [19]:
# 출간일, 쪽수, 역자
for sub_li in sub_lis:
    text = sub_li.get_text()
    if '출간 :' in text:
        date = int(re.sub('[^0-9]','', text))
    elif '페이지 :' in text:
        page = int(re.sub('[^0-9]','', text))
    elif '번역 :' in text:
        trans = text.split(':')[1].strip()
print(date, page, trans)

20220420 388 한선용


In [28]:
from tqdm.notebook import tqdm

In [29]:
lines = []
for page in tqdm(range(1, 11)):
    site = 'https://www.hanbit.co.kr'
    url = site + f'/store/books/new_book_list.html?page={page}&brand=&cate1=&cate2=&searchKey=&keyWord='
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        title = li.select_one('.book_tit').get_text().strip()
        author = li.select_one('.book_writer').get_text().strip()
        
        href = li.select_one('.book_tit').find('a')['href']
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
        sub_lis = sub_soup.select('.info_list > li')
        trans = '-'
        for sub_li in sub_lis:
            text = sub_li.get_text()
            if '출간 :' in text:
                date = int(re.sub('[^0-9]','', text))
            elif '페이지 :' in text:
                page = int(re.sub('[^0-9]','', text))
            elif '번역 :' in text:
                trans = text.split(':')[1].strip()
        line = [title, author, trans, date, page]
        lines.append(line)

  0%|          | 0/10 [00:00<?, ?it/s]

In [31]:
df = pd.DataFrame(lines, columns=['제목', '작가', '번역', '출간', '페이지'])
df

,제목,작가,번역,출간,페이지
0,엑셀이 편해지는 파이썬,펠릭스 춤슈타인,한선용,20220420,388
1,올림포스 연대기,김재훈,-,20220420,280
2,쉽게 배우는 AWS AI 서비스,"피터 엘거 , 오언 셔너히","맹윤호 , 임지순",20220415,396
3,상식적으로 상식을 배우는 법,제바스티안 클루스만,이지윤,20220414,248
4,엑셀로 부자되기,김태형,-,20220412,312
...,...,...,...,...,...
195,"풀스택 서버리스 : 리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발",네이더 다빗,김범준,20210705,216
196,한 권으로 배우는 작고 예쁜 꽃자수,장미나(미나랜드),-,20210705,240
197,"IT CookBook, 처음 만나는 회로이론(2판)",방성완,-,20210702,400
198,안전필수 시스템 제어 설계,"김국헌 , 김태효 , 이성섭",-,20210702,488


In [32]:
df.to_csv('한빛.csv', index=False)